Блокнот по видео [Diffusion models from scratch in PyTorch](https://youtu.be/a4Yfz2FxXiY?si=ONoYp-w4-qlJYd5G)

Изучаю датасет

Как датасет там дан StanfordCars Dataset - встроенный в PyTorch, который содержит 8000 фотографий машин

In [1]:
import torch
import torchvision
import matplotlib.pyplot as plt

def show_images(dataset, num_samples = 20, cols = 4):
    '''Plots some samples from dataset'''

    plt.figure(figsize=(15,15))
    for i, img in enumerate(dataset):
        if i == num_samples:
            break
        plt.subplot(int(num_samples/cols) + 1, i + 1)
        plt.imshow(img[0])

data = torchvision.datasets.StanfordCars(root='cars', download=True)
show_images(data)

ValueError: The original URL is broken so the StanfordCars dataset cannot be downloaded anymore.

In [ ]:
import requests
import os
import time
import json
from tqdm import tqdm  # Для прогресс-бара (pip install tqdm)

class DanbooruParser:
    def __init__(self, tags: str, output_dir: str = "downloaded", 
                 limit_per_page: int = 100, max_pages: int = 5, 
                 min_score: int = 20, rating: str = "g", 
                 delay: float = 1.0):
        """
        :param tags: Теги для поиска (например "ayanami_rei")
        :param output_dir: Папка для сохранения
        :param limit_per_page: Постов на страницу (max 200)
        :param max_pages: Максимальное количество страниц
        :param min_score: Минимальный рейтинг поста для загрузки
        :param rating: Рейтинг контента (g, s, q, e)
        :param delay: Задержка между запросами (секунды)
        """
        self.tags = tags
        self.output_dir = output_dir
        self.limit = min(limit_per_page, 200)  # Danbooru ограничивает 200
        self.max_pages = max_pages
        self.min_score = min_score
        self.rating = rating
        self.delay = delay
        self.base_url = "https://danbooru.donmai.us/posts.json"
        self.metadata = []
        os.makedirs(output_dir, exist_ok=True)

    def fetch_page(self, page: int):
        """Загрузка страницы с постами"""
        params = {
            "tags": self.tags,
            "page": page,
            "limit": self.limit,
            "rating": self.rating
        }
        try:
            response = requests.get(self.base_url, params=params)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Ошибка при загрузке страницы {page}: {str(e)}")
            return None

    def download_image(self, url: str, filename: str):
        """Скачивание и сохранение изображения"""
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(8192):
                    f.write(chunk)
            return True
        except Exception as e:
            print(f"Ошибка загрузки {url}: {str(e)}")
            return False

    def save_metadata(self):
        """Сохранение метаданных в JSON"""
        with open(os.path.join(self.output_dir, "metadata.json"), "w") as f:
            json.dump(self.metadata, f, indent=2)

    def run(self):
        """Основной цикл парсинга"""
        print(f"🚀 Начало парсинга тега: {self.tags}")
        downloaded_ids = set()
        total_downloaded = 0

        for page in range(1, self.max_pages + 1):
            print(f"📖 Страница {page}/{self.max_pages}")
            data = self.fetch_page(page)
            if not data:
                break

            if not data:  # Нет данных на странице
                print("ℹ️ Достигнут конец результатов")
                break

            for post in tqdm(data, desc="Скачивание"):
                # Проверка условий
                if post["id"] in downloaded_ids:
                    continue
                if post["score"] < self.min_score:
                    continue
                if not post.get("file_url"):
                    continue

                # Подготовка данных
                file_ext = os.path.splitext(post["file_url"])[1]
                filename = f"{post['id']}{file_ext}"
                filepath = os.path.join(self.output_dir, filename)

                # Скачивание
                if self.download_image(post["file_url"], filepath):
                    downloaded_ids.add(post["id"])
                    total_downloaded += 1
                    
                    # Сохранение метаданных
                    metadata_entry = {
                        "id": post["id"],
                        "tags": post["tag_string"],
                        "score": post["score"],
                        "rating": post["rating"],
                        "source": post.get("source", ""),
                        "file_path": filename
                    }
                    self.metadata.append(metadata_entry)

            time.sleep(self.delay)  # Защита от бана

        self.save_metadata()
        print(f"✅ Готово! Скачано изображений: {total_downloaded}")
        print(f"💾 Метаданные сохранены в metadata.json")

if __name__ == "__main__":
    # Настройки парсера
    parser = DanbooruParser(
        tags="ayanami_rei", 
        output_dir="rei_images",
        limit_per_page=100,   # 100 постов на страницу
        max_pages=10,         # Макс 10 страниц (1000 изображений)
        min_score=5,         # Минимальный рейтинг поста
        rating="g",           # Только безопасный контент
        delay=3             # Задержка 1.5 сек между запросами
    )
    parser.run()

🚀 Начало парсинга тега: ayanami_rei
📖 Страница 1/10


In [6]:
data

[{'id': 9721849,
  'created_at': '2025-07-31T19:24:26.924-04:00',
  'uploader_id': 786597,
  'score': 3,
  'source': 'https://i.pximg.net/img-original/img/2025/06/02/17/08/44/131097568_p0.png',
  'md5': '640fd372f7d95d3b49a47c45bbd04a68',
  'last_comment_bumped_at': None,
  'rating': 's',
  'image_width': 3521,
  'image_height': 2093,
  'tag_string': '1girl absurdres akatsukireii alternate_costume ayanami_rei blue_hair breasts character_name cleavage_cutout closed_mouth clothing_cutout commentary_request cooking_pot dishwasher dress electric_kettle expressionless feet_out_of_frame highres indoors kettle kitchen kitchen_knife knife knife_block looking_at_viewer medium_breasts neon_genesis_evangelion plate plate_stack red_eyes refrigerator short_hair sink sitting sleeveless sleeveless_dress solo spice_rack stove tile_wall tiles two-tone_dress window',
  'fav_count': 2,
  'file_ext': 'png',
  'last_noted_at': None,
  'parent_id': None,
  'has_children': False,
  'approver_id': None,
  'ta